In [ ]:
!git clone https://github.com/GewelsJI/SINet-V2.git
!pip install torch torchvision pillow opencv-python numpy tensorboardX --default-timeout 1000
%cd SINet-V2

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown -q https://drive.google.com/uc?id=1V0iSEdYJrT0Y_DHZfVGMg6TySFRNTy4o -O COD-TestDataset.zip
!unzip -q COD-TestDataset.zip -d ./ && rm COD-TestDataset.zip

!mkdir -p "./Dataset/TestDataset/"
!cp -r COD-TestDataset/* ./Dataset/TestDataset/
!rm -rf COD-TestDataset

!gdown -q https://drive.google.com/uc?id=1M8-Ivd33KslvyehLK9_IUBGJ_Kf52bWG -O COD-TrainDataset.zip
!unzip -q COD-TrainDataset.zip -d ./ && rm COD-TrainDataset.zip

!mkdir -p "./Dataset/TrainValDataset/"
!cp -r COD-TrainDataset/* ./Dataset/TrainValDataset/
!rm -rf COD-TrainDataset

!mkdir -p "./snapshot/SINet_V2/"
!gdown -q https://drive.google.com/uc?id=1D3RKQ8Nzd0ArV_c47StVKEuaoYTwnclR -O ./snapshot/SINet_V2/Net_epoch_best.pth

!rm -rf _MACOSX

In [ ]:
!gdown -q https://drive.google.com/uc?id=1QumnqSY_2wa-81-Ti0X1-jQzaGDIfa9r -O ./snapshot/res2net50_v1b_26w_4s-3cf99910.pth

In [ ]:
import fileinput

file_path = '/kaggle/working/SINet-V2/MyTesting.py'
target_line = "misc.imsave(save_path+name, res)"
replacement_line = "cv2.imwrite(save_path+name, res*255)"

with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
    for line in file:
        if target_line in line:
            print(f"# {line.strip()}")  
            print(replacement_line)      
        else:
            print(line, end='')          

In [ ]:
!pip install thop

In [ ]:
contents = """
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torch
import torch.nn.functional as F

__all__ = ['Res2Net', 'res2net50_v1b', 'res2net101_v1b', 'res2net50_v1b_26w_4s']

model_urls = {
    'res2net50_v1b_26w_4s': 'https://shanghuagao.oss-cn-beijing.aliyuncs.com/res2net/res2net50_v1b_26w_4s-3cf99910.pth',
    'res2net101_v1b_26w_4s': 'https://shanghuagao.oss-cn-beijing.aliyuncs.com/res2net/res2net101_v1b_26w_4s-0812c246.pth',
}


class Bottle2neck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, baseWidth=26, scale=4, stype='normal'):
        super(Bottle2neck, self).__init__()

        width = int(math.floor(planes * (baseWidth / 64.0)))
        self.conv1 = nn.Conv2d(inplanes, width * scale, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width * scale)

        if scale == 1:
            self.nums = 1
        else:
            self.nums = scale - 1
        if stype == 'stage':
            self.pool = nn.AvgPool2d(kernel_size=3, stride=stride, padding=1)
        convs = []
        bns = []
        for i in range(self.nums):
            convs.append(nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, bias=False))
            bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)

        self.conv3 = nn.Conv2d(width * scale, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stype = stype
        self.scale = scale
        self.width = width

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        spx = torch.split(out, self.width, 1)
        for i in range(self.nums):
            if i == 0 or self.stype == 'stage':
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.relu(self.bns[i](sp))
            if i == 0:
                out = sp
            else:
                out = torch.cat((out, sp), 1)
        if self.scale != 1 and self.stype == 'normal':
            out = torch.cat((out, spx[self.nums]), 1)
        elif self.scale != 1 and self.stype == 'stage':
            out = torch.cat((out, self.pool(spx[self.nums])), 1)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Res2Net(nn.Module):

    def __init__(self, block, layers, baseWidth=26, scale=4, num_classes=1000):
        self.inplanes = 64
        super(Res2Net, self).__init__()
        self.baseWidth = baseWidth
        self.scale = scale
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1, 1, bias=False)
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.AvgPool2d(kernel_size=stride, stride=stride,
                             ceil_mode=True, count_include_pad=False),
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample=downsample,
                            stype='stage', baseWidth=self.baseWidth, scale=self.scale))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, baseWidth=self.baseWidth, scale=self.scale))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)    # torch.Size([1, 64, 176, 176])
        x = self.maxpool(x)

        x = self.layer1(x)  # torch.Size([1, 256, 88, 88])
        x = self.layer2(x)  # torch.Size([1, 512, 44, 44])
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def res2net50_v1b(pretrained=False, **kwargs):
    model = Res2Net(Bottle2neck, [3, 4, 6, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net50_v1b_26w_4s']))
    return model


def res2net101_v1b(pretrained=False, **kwargs):
    model = Res2Net(Bottle2neck, [3, 4, 23, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net101_v1b_26w_4s']))
    return model


def res2net50_v1b_26w_4s(pretrained=False, **kwargs):
    model = Res2Net(Bottle2neck, [3, 4, 6, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        # model_state = torch.load('/media/nercms/NERCMS/GepengJi/Medical_Seqmentation/CRANet/models/res2net50_v1b_26w_4s-3cf99910.pth')
        model_state = torch.load('./snapshot/res2net50_v1b_26w_4s-3cf99910.pth')
        model.load_state_dict(model_state)
        # model.load_state_dict(model_zoo.load_url(model_urls['res2net50_v1b_26w_4s']))
    return model


def res2net101_v1b_26w_4s(pretrained=False, **kwargs):
    model = Res2Net(Bottle2neck, [3, 4, 23, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net101_v1b_26w_4s']))
    return model


def res2net152_v1b_26w_4s(pretrained=False, **kwargs):
    model = Res2Net(Bottle2neck, [3, 8, 36, 3], baseWidth=26, scale=4, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['res2net152_v1b_26w_4s']))
    return model


if __name__ == '__main__':
    images = torch.rand(1, 3, 352, 352).cuda(0)
    model = res2net50_v1b_26w_4s(pretrained=False)
    model = model.cuda(0)
    print(model(images).size())

"""

with open('/kaggle/working/SINet-V2/lib/Res2Net_v1b.py', 'w') as file:
    file.write(contents)

In [ ]:
!python MyTrain_Val.py

In [ ]:
test_contents = """
import torch
import torch.nn.functional as F
import numpy as np
import os, argparse
import cv2
from lib.Network_Res2Net_GRA_NCD import Network
from utils.data_val import test_dataset
from sklearn.metrics import jaccard_score
import warnings

warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser()
parser.add_argument('--testsize', type=int, default=352, help='testing size')
parser.add_argument('--pth_path', type=str, default='./snapshot/SINet_V2/Net_epoch_best.pth')
opt = parser.parse_args()

# List to store IoU scores for each image
iou_scores = []

for _data_name in ['COD10K']:
    data_path = './Dataset/TestDataset/{}/'.format(_data_name)
    save_path = './res/{}/{}/'.format(opt.pth_path.split('/')[-2], _data_name)
    model = Network(imagenet_pretrained=False)
    model.load_state_dict(torch.load(opt.pth_path))
    model.cuda()
    model.eval()

    os.makedirs(save_path, exist_ok=True)
    image_root = '{}/Imgs/'.format(data_path)
    gt_root = '{}/GT/'.format(data_path)
    test_loader = test_dataset(image_root, gt_root, opt.testsize)

    for i in range(test_loader.size):
        image, gt, name, _ = test_loader.load_data()
        gt = np.asarray(gt, np.float32)
        gt /= (gt.max() + 1e-8)
        image = image.cuda()

        res5, res4, res3, res2 = model(image)
        res = res2
        res = F.upsample(res, size=gt.shape, mode='bilinear', align_corners=False)
        res = res.sigmoid().data.cpu().numpy().squeeze()
        res = (res - res.min()) / (res.max() - res.min() + 1e-8)
        pred_binary = (res > 0.5).astype(np.uint8).flatten()
        gt_binary = (gt > 0.5).astype(np.uint8).flatten()
        
        iou = jaccard_score(gt_binary, pred_binary, average='binary')
        iou_scores.append(iou)
        
        print('> {} - {} | IoU: {:.4f}'.format(_data_name, name, iou))
        cv2.imwrite(save_path + name, res * 255)

# Calculate and report IoU statistics
average_iou = np.mean(iou_scores)
best_iou = np.max(iou_scores)
worst_iou = np.min(iou_scores)
std_dev_iou = np.std(iou_scores)

print("\\nSummary of IoU Scores:")
print(f"Average IoU: {average_iou:.4f}")
print(f"Best IoU: {best_iou:.4f}")
print(f"Worst IoU: {worst_iou:.4f}")
print(f"Standard Deviation of IoU: {std_dev_iou:.4f}")
"""

with open('/kaggle/working/SINet-V2/MyTesting.py', 'w') as file:
    file.write(test_contents)

In [ ]:
!python MyTesting.py